In [34]:
import os
import pandas as pd
import numpy as np
import json
import itertools
from tqdm import tqdm
import pickle

### Video List

In [52]:
video_df = pd.read_csv("/Users/akospeter/Downloads/video-gait-v1/annotations/video_list.csv")

In [53]:
video_df

In [55]:
video_df.groupby('Video_File').count()

### Surginfo

In [14]:
surginfo_df = pd.read_csv("/Users/akospeter/Downloads/video-gait-v1/annotations/surgInfo.csv")

In [15]:
surginfo_df

### AllData

In [16]:
alldata_df = pd.read_csv("/Users/akospeter/Downloads/video-gait-v1/annotations/alldata.csv")

In [17]:
alldata_df

In [18]:
alldata_df.columns

In [19]:
exam_ids_video = video_df['Exam_ID']
len(exam_ids_video)

In [20]:
exam_ids_alldata = alldata_df['examid']
len(exam_ids_alldata)

In [21]:
common_exam_ids = [curr for curr in exam_ids_video if curr in exam_ids_alldata]
len(common_exam_ids)

In [22]:
video_id = []
for exam_id in alldata_df['examid'].tolist():
    video_file = video_df.loc[video_df['Exam_ID'] == exam_id, 'Video_File'].values
    if len(video_file) > 1:
        print(video_file)
        video_id.append(video_file[0])
    elif len(video_file) == 1:
        video_id.append(video_file[0])
    else:
        print(f"No video for {exam_id}")
        video_id.append(np.nan)

In [23]:
len(video_id)

In [24]:
alldata_df['video_id'] = video_id

In [25]:
alldata_df = alldata_df.dropna()

In [44]:
alldata_df

In [56]:
video_id_predicted_values = alldata_df[['video_id', 'side', 'KneeFlex_maxExtension', 'speed', 'cadence', 'steplen']]

In [57]:
video_id_predicted_values.to_csv("video_id_predicted_values.csv", index=False)

In [111]:
def convert_json2csv(video_id,json_dir):
    resL = np.zeros((1500,75))
    resL[:] = np.nan
    for frame in range(1,1500):
        videoid_str = '%s-processed' % (video_id)
        test_image_json = '%s%s/%s_%s_keypoints.json' %\
            (json_dir, videoid_str, videoid_str,str(frame).zfill(12))

        if not os.path.isfile(test_image_json):
            break
        with open(test_image_json) as data_file:  
            data = json.load(data_file)
        for person in data['people']:
            keypoints = person['pose_keypoints_2d']
            xcoords = [keypoints[i] for i in range(len(keypoints)) if i % 3 == 0]
            counter = 0
            if np.max(xcoords) < 320:
                if(counter > 0):
                    print("duplicate person on left half of video %s") % video_id
                    print(video_id)
                counter += 1
                resL[frame-1,:] = keypoints

    #we can save space by dropping rows after the last row that isn't all nan
    check = np.apply_along_axis(lambda x: np.any(~np.isnan(x)),1,resL)
    for i in range(len(check)-1,-1,-1):
        if check[i]:
            break
    return resL[:i+1] # resR[:i+1]

In [198]:
res = []
actual_video_ids = os.listdir(f"../data/video-gait-v1/openpose/")
actual_video_ids = [actual_video_id.split('-')[0] for actual_video_id in actual_video_ids]
for video_id in tqdm(actual_video_ids):
    frames = os.listdir(f"../data/video-gait-v1/openpose/{video_id}-processed/")
    for frame in frames:
        json_path = f"../data/video-gait-v1/openpose/{video_id}-processed/{frame}"
        try:
            with open(json_path) as data_file:  
                data = json.load(data_file)
            for person in data['people']:
                keypoints = person['pose_keypoints_2d']
                xcoords = [keypoints[i] for i in range(len(keypoints)) if i % 3 == 0]
                counter = 0
                if np.max(xcoords) < 320:
                    if(counter > 0):
                        print(f"duplicate person on left half of video {video_id}")
                        print(video_id)
                    counter += 1
                    curr_res = [video_id, frame] + keypoints
                    res.append(curr_res)
        except Exception as e:
            print(f"An unexpected error occurred with file {json_path}: {e}")

In [201]:
videos_data = pd.DataFrame(res)

In [202]:
videos_data

In [207]:
videos_data.groupby(0).count()[1].describe()

In [215]:
we_have_video_ids = set(videos_data[0].tolist())
print(len(set(we_have_video_ids)))
all_video_ids = alldata_df['video_id'].tolist()
print(len(all_video_ids))

In [216]:
both = [video_id for video_id in we_have_video_ids if video_id in all_video_ids]

In [217]:
len(both)

In [220]:
filtered_videos_data = videos_data[videos_data[0].isin(both)]

In [2]:
body_parts = [ "Nose","Neck","RShoulder","RElbow","RWrist","LShoulder","LElbow","LWrist","MidHip","RHip","RKnee","RAnkle","LHip","LKnee","LAnkle","REye","LEye","REar","LEar","LBigToe","LSmallToe","LHeel","RBigToe","RSmallToe","RHeel"]
columns = ['video_id', 'frame'] + [f"{body_part}_{elem}" for body_part in body_parts for elem in ['x', 'y', 'prob']]

In [241]:
filtered_videos_data.columns = columns

In [244]:
filtered_videos_data

In [245]:
filtered_videos_data.to_csv("features.csv", index=False)

In [5]:
filtered_videos_data = pd.read_csv("features.csv")
filtered_videos_data

In [7]:
video_sizes = filtered_videos_data.groupby("video_id").count()['frame']

In [9]:
video_sizes.describe()

In [51]:
we_have_video_ids = set(filtered_videos_data['video_id'].tolist())
print(len(set(we_have_video_ids)))
all_video_ids = alldata_df['video_id'].tolist()
print(len(all_video_ids))

In [29]:
JSON_DIR = '../data/video-gait-v1/openpose/'

In [30]:
existing_csvs = os.listdir("../data/processed/video_csvs/")
existing_csvs = [e.split('_')[0] for e in existing_csvs]

In [31]:
video_ids = alldata_df['video_id'].tolist()

In [32]:
for i,video_id in enumerate(video_ids):
    if(i % 100 == 0):
        print("%.2f percent done" % (i*100./len(video_ids)))
    if str(video_id) not in existing_csvs:
        resL = convert_json2csv(video_id,JSON_DIR)
        save_string_L = '%s/processed/video_csvs/%s.csv' % ('../data',video_id)
        np.savetxt(save_string_L, resL, delimiter=',')

### Combine them

In [33]:
import pandas as pd
import pickle
import numpy as np

all_videos = {}
for i,video_id in enumerate(video_ids):
    if(i % 100 == 0):
        print("%.2f percent done" % (i*100./len(video_ids)))
    file_string = '%s/processed/video_csvs/%s.csv' % ('../data',video_id)
    try: 
        all_videos[video_id] = np.loadtxt(file_string, delimiter=',')
    except:
        print("missing %s" % (video_id))

In [35]:
with open('../data/processed/all_videos_dict.pickle', 'wb') as handle:
    pickle.dump(all_videos, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [36]:
alldata_processed = alldata_df.groupby(['video_id'],as_index=False).head(1)

In [37]:
alldata_processed

### Utils

In [148]:
NOSE = 0
NECK = 1
RSHO = 2
RELB = 3
RWRI = 4
LSHO = 5
LELB = 6
LWRI = 7
MHIP = 8
RHIP = 9
RKNE = 10
RANK = 11
LHIP = 12
LKNE = 13
LANK = 14
REYE = 15
LEYE = 16
REAR = 17
LEAR = 18
LBTO = 19
LSTO = 20
LHEL = 21
RBTO = 22
RSTO = 23
RHEL = 24

In [158]:
def expand_columns(col_lst):
    return flatten([[2*x,2*x+1] for x in col_lst])

def max_pct_nan_or_zero_given_cols(A,col_lst):
    col_indices_used = expand_columns(col_lst)
    A = A.copy()[:,col_indices_used]
    mask = (A==0) | (np.isnan(A))
    return np.max(np.sum(mask,axis=0))*1.0/len(A)

def drop_confidence_cols(res):
    res = res.copy()
    num_parts = res.shape[1]/3
    processed_cols = [True,True,False] * int(num_parts)
    return res[:,processed_cols]
    
def flatten(lst):
    return list(itertools.chain.from_iterable(lst))

In [159]:
COLS_USED = [LANK,RANK,LKNE,RKNE,LHIP,RHIP,LBTO,RBTO]

### Filtering

In [160]:
result = []
vid_length = 124
for videoid, raw_video in all_videos.items():
    if len(raw_video.shape) == 2 and len(raw_video) >= vid_length:
        raw_video = drop_confidence_cols(raw_video[:500,:].copy())
        #for each column, compute the % of values missing
        #then, take the max of those missing values
        pct_missing = max_pct_nan_or_zero_given_cols(raw_video,COLS_USED)
        
        n_segments = 0
        start_idx = 0
        for i in range(start_idx,500-vid_length,31):
            raw_video_chunk = raw_video[i:i+vid_length,:]
            pct_missing_chunk =\
                max_pct_nan_or_zero_given_cols(raw_video_chunk,COLS_USED)
            if pct_missing_chunk <= 0.25 and len(raw_video_chunk) == vid_length:
                n_segments += 1
        
        result.append([videoid,pct_missing,n_segments])
result = np.array(result)

In [175]:
df = pd.DataFrame(np.array(result),columns=['video_id','pct_missing','n_segments'])
df['pct_missing'] = df['pct_missing'].astype(float)
df = df[(df['pct_missing'] <= 0.25)]
df = df.merge(right=alldata_processed[['video_id','Patient_ID']],on=['video_id'],how='inner')

In [176]:
df

In [181]:
curr = all_videos['16397602']

In [183]:
curr.shape

In [41]:
raw_asda = all_videos['10312701']

In [43]:
raw_asda.shape

In [6]:
import pickle

In [7]:
X_FILENAME = "../data/X_100.pkl"

In [8]:
with open(X_FILENAME, 'rb') as file:
    X = pickle.load(file)

In [ ]:
max_val =
for video_id, X_data in X:
    